# Train Translator

Based on: https://huggingface.co/docs/transformers/tasks/translation

In [1]:
import sys, os, datetime
import json
import torch
import random
from tqdm.notebook import tqdm
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import TranslationPipeline
from datasets import load_dataset, Dataset
import cdli

In [2]:
!ls "../results"

t5-base-akkelxsux-en-20220719-204245	t5-base-p-akksux-en-20220721-232944
t5-base-akkelxsux-en-20220719-204449	t5-base-p-akksux-en-20220721-233149
t5-base-akk-sux-20220719-013158		t5-base-p-akksux-en-20220722-122042
t5-base-akk-sux-20220719-013538		t5-base-p-akksux-en-20220722-122256
t5-base-akk-sux-20220719-014641		t5-base-p-akksux-en-20220722-173018
t5-base-akk-sux-20220719-024752		t5-small-akk-sux
t5-base-akk-sux-20220719-103633		t5-small-akkts-suxts
t5-base-akk-sux-20220719-112018		t5-small-p-akksux-en-20220722-094651
t5-base-akk-sux-20220719-122111		t5-small-sux-akk
t5-base-bi-akk-elx-sux-20220719-190709	transliteration-merges.txt
t5-base-bi-akk-sux-20220719-131838	transliteration-tokenizer
t5-base-bi-akk-sux-20220719-133120	transliteration_tokenizer.json
t5-base-bi-p-akksux-en-20220722-005735	transliteration-vocab.json
t5-base-bi-p-akksux-en-20220723-023520	ttok
t5-base-p-akksux-en-20220721-231405


In [2]:
os.environ["WANDB_NOTEBOOK_NAME"] = "TrainTranslator.ipynb"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

source_langs = set(["akk", "sux"])

# target_langs = set(["en", "it", "es", "fr", "de"])
target_langs = set(["en"])

base_model_id = "t5-base"

model_max_length = 512
batch_size = 8 if base_model_id == "t5-base" else 128

num_train_epochs = 30

is_bi = True
paragraphs = True

In [3]:
date_id = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
flags = ""
if is_bi:
    flags += "-bi"
if paragraphs:
    flags += "-p"
model_id = f"{base_model_id}{flags}-{''.join(sorted(list(source_langs)))}-{''.join(sorted(list(target_langs)))}-{date_id}"
model_id

't5-base-bi-p-akksux-en-20220723-023520'

In [4]:
has_cuda = torch.cuda.is_available()
device = torch.cuda.device(0) if has_cuda else "cpu"
has_cuda, device

(True, <torch.cuda.device at 0x7f419c71a6b0>)

In [5]:
!nvidia-smi

Sat Jul 23 02:35:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 30%   34C    P8    29W / 350W |    168MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load Training Data

In [6]:
avg_src_chars_per_token = 1.8713256996006793
avg_tgt_chars_per_token = 2.577806274115267

In [7]:
lang_full = {
    "akk": "Akkadian",
    "elx": "Elamite",
    "sux": "Sumerian",
    "akkts": "Akkadian",
    "elxts": "Elamite",
    "suxts": "Sumerian",
    "en": "English",
    "it": "Italian",
    "es": "Spanish",
    "fr": "French",
    "de": "German",
}

In [8]:
def get_prefix(src_lang, tgt_lang):
    s = lang_full[src_lang]
    t = lang_full[tgt_lang]
    return f"translate {s} to {t}: "
    
get_prefix("suxts", "es")

'translate Sumerian to Spanish: '

In [9]:
replacements = [
    ("ā", "a"),
    ("Ā", "a"),
    ("ḫ", "h"),
    ("Ḫ", "H"),
    ("ī", "i"),
    ("Ī", "I"),
#     ("î", "i"),
#     ("Î", "I"),
    ("ř", "r"),
    ("Ř", "R"),
    ("š", "sh"),
    ("Š", "Sh"),
    ("ṣ", "sh"),
    ("Ṣ", "Sh"),
    ("ṭ", "t"),
    ("Ṭ", "T"),
    ("ū", "u"),
    ("Ū", "U"),
]
def replace_unsupported(text):
    r = text
    for s, t in replacements:
        r = r.replace(s, t)
    return r

In [10]:
publications = cdli.get_atf()

Parsing atf


In [11]:
len(publications), "publications"

(134712, 'publications')

In [12]:
def target_ok(target_text):
    if len(target_text) == 0:
        return False
    if len(set(target_text.replace(" ", ""))) < 2:
        return False
    return True
    

def test_target_ok(text):
    ok = target_ok(text)
    print(ok, repr(text))
    
test_target_ok("")
test_target_ok(" ")
test_target_ok("xx xxx x")
test_target_ok(".. . .. ")
test_target_ok("Hi")

False ''
False ' '
False 'xx xxx x'
False '.. . .. '
True 'Hi'


In [13]:
new_sourceandtargets = []

added_sources = set()

wmax_num_tokens = model_max_length - 192

def wrap_paragraph(paragraph, lines, src_lang, tgt_lang):
    ptag, pline_start_index, pline_end_index = paragraph
    wline_ranges = []
    wline_tok_len = 0.0
    
    def start_new_line(pline_index):
#         print("start", pline_index)
        wline_ranges.append((pline_index, pline_index + 1))
        
    def append_line(pline_index):
#         print("append", pline_index)
        r = wline_ranges[-1]
        if r[1] == pline_index:
            wline_ranges[-1] = (r[0], r[1] + 1)
        else:
            print(f"Missing line: got {pline_index}, expected {r[1]}: {wline_ranges}")

    for pline_index in range(pline_start_index, pline_end_index):
        pline_num_toks = len(lines[pline_index].text) / avg_src_chars_per_token + 1.0
        if len(wline_ranges) == 0 or (wline_tok_len + pline_num_toks > wmax_num_tokens):
            start_new_line(pline_index)
            wline_tok_len = 0.0
        else:
            append_line(pline_index)
        wline_tok_len += pline_num_toks
    return wline_ranges

def add_line_ranges(area, b, e):
#                     print("-"*50)
    ls = " ".join([x.text for x in area.lines[b:e]])
    ls = " ".join(ls.split(" "))
    prefixed_ls = st_prefix + ls
    if prefixed_ls in added_sources:
        return
    lt = " ".join([(x.languages[t] if t in x.languages else "") for x in area.lines[b:e]])
    lt = " ".join(lt.split(" "))
    lt = replace_unsupported(lt)
    if not target_ok(lt):
        return
#                     print(ls)
#                     print(lt)
    added_sources.add(prefixed_ls)
    new_sourceandtargets.append((prefixed_ls, lt))
    if is_bi:
        new_sourceandtargets.append((ts_prefix + lt, ls))



for s in source_langs:    
    for t in target_langs:
        print("Preparing", s, "to", t)
        st_prefix = get_prefix(s, t)
        ts_prefix = get_prefix(t, s)
        for pub in tqdm([p for p in publications if p.language==s]):
            for area in pub.text_areas:
                if not any(x for x in area.lines if t in x.languages):
                    continue
                if paragraphs:
                    paragraphs = area.lines_to_paragraphs(s)
                    line_ranges = []                
                    for p in paragraphs:                    
                        wlines = wrap_paragraph(p, area.lines, s, t)
                        line_ranges.extend(wlines)
    #                 print("="*50, len(area.lines))
                    for b, e in line_ranges:
                        add_line_ranges(area, b, e)
                for i, _ in enumerate(area.lines):
                    add_line_ranges(area, i, i + 1)

random.shuffle(new_sourceandtargets)
new_all_translations = Dataset.from_dict({"source": [x[0] for x in new_sourceandtargets], "target": [x[1] for x in new_sourceandtargets]})
new_all_translations

Preparing sux to en


  0%|          | 0/99456 [00:00<?, ?it/s]

Preparing akk to en


  0%|          | 0/21820 [00:00<?, ?it/s]

Dataset({
    features: ['source', 'target'],
    num_rows: 143396
})

In [14]:
all_translations = new_all_translations

In [15]:
all_translations[1120:1200]

{'source': ["translate English to Sumerian: {a time when anyone is to go where they will, to hurl no insults at one's fellow,}",
  'translate Sumerian to English: nam-<gi4>-me-esz3-a-ni-ta',
  'translate Sumerian to English: gu3-de2-a-ar inim {d}nin-gir2-su-ka uri3-am3 mu-du3',
  'translate English to Sumerian: Destroyer of the mountain, she unleashed the arrow towards ... by means of (her) power, who smote(?) with the arm(?) in the mountain',
  'translate English to Sumerian: my tablet I received, ...;',
  'translate Akkadian to English: szum-ma _nin-dingir lukur_',
  'translate English to Akkadian: a pure residence,',
  'translate English to Sumerian: 28 gur, regular offerings of Zabala, of long-headed (grain),',
  'translate Sumerian to English: nig2 e3',
  'translate English to Sumerian: give him water to drink!',
  'translate English to Sumerian: also for her own life,',
  'translate Akkadian to English: ar-ba-um',
  'translate Sumerian to English: [u4 x] 2(disz)-kam',
  'translat

In [16]:
translations = all_translations.train_test_split(test_size=0.1)
translations

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 129056
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 14340
    })
})

In [17]:
original_tests = translations["test"]
original_tests

Dataset({
    features: ['source', 'target'],
    num_rows: 14340
})

In [18]:
test_starts = [f"translate {lang_full[s]} to " for s in source_langs]
print(test_starts)

def should_test(t):
    return any(t["source"].startswith(s) for s in test_starts)

translations["test"] = original_tests.filter(should_test)
translations["test"]

Parameter 'function'=<function should_test at 0x7f40b754f370> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


['translate Sumerian to ', 'translate Akkadian to ']


  0%|          | 0/15 [00:00<?, ?ba/s]

Dataset({
    features: ['source', 'target'],
    num_rows: 7050
})

## Tokenize the Data

In [19]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, model_max_length=model_max_length)

In [20]:
tokenizer.model_max_length

512

In [21]:
print("pad", tokenizer.pad_token, tokenizer.pad_token_id)
print("eos", tokenizer.eos_token, tokenizer.eos_token_id)
print("unk", tokenizer.unk_token, tokenizer.unk_token_id)

pad <pad> 0
eos </s> 1
unk <unk> 2


In [22]:
ccc = 0
sum_src_chars_per_token = 0.0
num_src_chars_per_token = 0
sum_tgt_chars_per_token = 0.0
num_tgt_chars_per_token = 0

def preprocess_function(examples):
    global ccc, sum_src_chars_per_token, sum_tgt_chars_per_token, num_src_chars_per_token, num_tgt_chars_per_token
#     print(examples)
    inputs = [example for example in examples["source"]]
    targets = [example for example in examples["target"]]
    model_inputs = tokenizer(inputs, max_length=model_max_length, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=model_max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    
    nexamples = len(inputs)
    for i in range(nexamples):
        nchar = len(inputs[i])
        ntoks = len(model_inputs["input_ids"][i])
        if ntoks > 0:
            sum_src_chars_per_token += nchar / ntoks
            num_src_chars_per_token += 1
        nchar = len(targets[i])
        ntoks = len(model_inputs["labels"][i])
        if ntoks > 0:
            sum_tgt_chars_per_token += nchar / ntoks
            num_tgt_chars_per_token += 1
    
    ccc += 1
    if ccc == 1:
        print(model_inputs["input_ids"][0])
        print(model_inputs["labels"][0])
        nchar = len(targets[0])
        ntoks = len(model_inputs["labels"][0])
        print(nchar, ntoks, nchar / ntoks)

    return model_inputs

tokenized_translations = translations.map(preprocess_function, batched=True)
tokenized_translations

  0%|          | 0/130 [00:00<?, ?ba/s]

[13959, 12198, 15, 5288, 12, 1566, 10, 3, 40, 1598, 450, 3, 2168, 18, 9, 122, 357, 1]
[8, 11479, 3, 40, 1598, 450, 12966, 15, 7, 7, 1]
27 11 2.4545454545454546


  0%|          | 0/8 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 129056
    })
    test: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7050
    })
})

In [23]:
avg_src_chars_per_token = sum_src_chars_per_token / num_src_chars_per_token
avg_tgt_chars_per_token = sum_tgt_chars_per_token / num_tgt_chars_per_token
print("avg_src_chars_per_token", "=", avg_src_chars_per_token)
print("avg_tgt_chars_per_token", "=", avg_tgt_chars_per_token)

avg_src_chars_per_token = 2.7019102175559038
avg_tgt_chars_per_token = 2.0653795753124626


In [24]:
tokenized_translations["train"] = tokenized_translations["train"].remove_columns(["source", "target"])
tokenized_translations["test"] = tokenized_translations["test"].remove_columns(["source", "target"])
tokenized_translations

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 129056
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7050
    })
})

In [25]:
source_max_length = max([len(x["input_ids"]) for x in tokenized_translations["train"]])
target_max_length = max([len(x["labels"]) for x in tokenized_translations["train"]])
source_max_length, target_max_length

(510, 503)

In [26]:
tokenized_translations["train"][0]["labels"][:10]

[8, 11479, 3, 40, 1598, 450, 12966, 15, 7, 7]

## Load the Model

In [27]:
model = AutoModelForSeq2SeqLM.from_pretrained(base_model_id, max_length=model_max_length)

In [28]:
model.config

T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 512,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to German: "
 

## Train

In [29]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
# data_collator

In [30]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"../results/{model_id}",
    evaluation_strategy="epoch",
    learning_rate=2*2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    fp16=has_cuda,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_translations["train"],
    eval_dataset=tokenized_translations["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)



Using amp half precision backend


In [31]:
trainer.train()

/home/fak/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 129056
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 483960
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: praeclarum. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.902100,2.351661
2,1.635100,2.056134
3,1.471000,1.884453
4,1.361100,1.776658
5,1.329900,1.689395
6,1.215600,1.624595
7,1.174500,1.576504
8,1.110700,1.535952
9,1.068900,1.495610
10,1.030500,1.458588


Saving model checkpoint to ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-500
Configuration saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-500/config.json
Model weights saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-1000
Configuration saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-1000/config.json
Model weights saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ../results/t5-bas

Saving model checkpoint to ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-7000
Configuration saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-7000/config.json
Model weights saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-7000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-7000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-7000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-5500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-7500
Configuration saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-7500/config.json
Model weights saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-7500/pytorch_model.bin
tokenizer config file saved in ../res

tokenizer config file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-13000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-13000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-11500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-13500
Configuration saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-13500/config.json
Model weights saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-13500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-13500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-13500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-1200

...

tokenizer config file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-465000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-465000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-463500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-465500
Configuration saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-465500/config.json
Model weights saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-465500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-465500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-465500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-p-akksux-en-20220723-023520/checkpo

Special tokens file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-471000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-469500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-471500
Configuration saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-471500/config.json
Model weights saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-471500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-471500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-471500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-470000] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-p-akksux-en-20220723-023520/che

Deleting older checkpoint [../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-475500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-477500
Configuration saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-477500/config.json
Model weights saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-477500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-477500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-477500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-476000] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-478000
Configuration saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-478000/config.json
Mod

Saving model checkpoint to ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-483500
Configuration saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-483500/config.json
Model weights saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-483500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-483500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-483500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-p-akksux-en-20220723-023520/checkpoint-482000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 7050
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=483960, training_loss=0.9971950555283212, metrics={'train_runtime': 63962.3927, 'train_samples_per_second': 60.531, 'train_steps_per_second': 7.566, 'total_flos': 5.571217012513997e+17, 'train_loss': 0.9971950555283212, 'epoch': 30.0})

## Sample

In [40]:
pipeline = TranslationPipeline(model=model.to("cpu"), tokenizer=tokenizer, max_length=model_max_length)

In [33]:
pipeline

In [34]:
pipeline("translate English to French: hello my name is Frank")

[{'translation_text': 'hu-mu-um dnanna-mu'}]

In [35]:
source_test = translations["test"][0]["source"]
target_test = translations["test"][0]["target"]
print(source_test)
print("-"*80)
print(target_test)

translate Sumerian to English: {d}nin#-[...] x x AN# [...]
--------------------------------------------------------------------------------
Ninhursag(?) ...


In [36]:
def translate(text):
    return pipeline(text)

translate(source_test)

[{'translation_text': 'Ningublaga, the lady of the mountain range, the mountain range of the mountain range, the mountain of the king, the mountain of the king, the mountain of the king, the mountain of the lands'}]

In [37]:
tests = original_tests
def sample(num_samples=100):
    for i in range(min(num_samples, tests.num_rows)):
        t = tests[i]
    #     print(t)
        src = t["source"]
        tgt = t["target"]
        query = src
        pred = pipeline(query)[0]["translation_text"]
        print("-"*48)
        print("QUERY ", query)
        print("TARGET", tgt)
        print("PRED  ", pred)
    #     break
    
sample()

------------------------------------------------
QUERY  translate English to Sumerian: The deliberations of the elders and juniors ... before ... the utterance(?) ...
TARGET [...] lu2#? banda3{+da} sza3 kusz2-u3-bi [...] igi-sze3 ka ba x [...]
PRED   ab-ba tur-tur-bi igi-sze3 [...] du11-ga x [...]
------------------------------------------------
QUERY  translate Sumerian to English: {d}nin#-[...] x x AN# [...]
TARGET Ninhursag(?) ...
PRED   Ningublaga, the lady of the mountain range, the mountain range of the mountain range, the mountain of the king, the mountain of the king, the mountain of the king, the mountain of the lands
------------------------------------------------
QUERY  translate Sumerian to English: [u4 ul4-li2-a-sze3] pa bi2-e3 %a an s,e-a-tim u2-szu-[pi2]
TARGET I (thus) made resplendent forever.
PRED   I made resplendent forever.
------------------------------------------------
QUERY  translate Akkadian to English: _e2-gal_ lu2-{d}marduk _lugal_
TARGET the palace of Amē

------------------------------------------------
QUERY  translate English to Akkadian: If a man ... grain ... gave, and then the grain into silver has converted, at harvest the grain and the interest on it, per 1 kor, 1 barig 4 ban2 he shall take
TARGET szum-ma _lu2 sze_-a-am a-na x x x id-di-in-ma _sze_-a-am a-na _ku3-babbar_ i-te-pu!-usz i-na e-bu-ri _sze_-a-am u3 _masz-bi 1(asz) gur 1(barig) 4(ban2)_ i-le-eq-qe2
PRED   szum-ma a-wi-lum _sze_ [...] id-di-in-ma _sze_ a-na _ku3-babbar_ u2-te-ep-pe2 i-na _buru14 sze_ u3 _masz2_-szu _1(asz) gur 1(barig) 4(ban2)_ i-le-qe2
------------------------------------------------
QUERY  translate Sumerian to English: abul 3(disz)-kam-ma ku4-ku4-da-ni-ta
TARGET When she entered the 3rd gate,
PRED   When she entered the 3rd gate,
------------------------------------------------
QUERY  translate Akkadian to English: _i7-i7_-szu
TARGET his rivers
PRED   his river
------------------------------------------------
QUERY  translate Sumerian to English: ki 

------------------------------------------------
QUERY  translate Sumerian to English: ki-la2-bi 1(disz) ma-na 1(u) 8(disz) gin2
TARGET Their weight: 1 ma-na, 18 shekels.
PRED   Its weight: 1 ma-na, 18 shekels.
------------------------------------------------
QUERY  translate Sumerian to English: lu2-mu igi im-mi-du8-am3
TARGET my own man saw it!
PRED   “Did you see my man?”
------------------------------------------------
QUERY  translate English to Sumerian: Shu-Erra, scribe, son of Ishar-beli, is your servant.
TARGET szu-er3-ra dub-sar dumu i-szar-be-li2 ARAD2-zu
PRED   szu-er3-ra dub-sar dumu i-szar-be-li2 ARAD2-zu
------------------------------------------------
QUERY  translate English to Sumerian: Basket-of-tablets: xxx xxx xxx
TARGET pisan-dub-ba nig2-ka9-ak sza3-bi su-ga sza3 ze2 i3-gal2
PRED   pisan-dub-ba nig2-ka9-ak sze erin2 szu-ku6 sza3 gu2-ab-baki
------------------------------------------------
QUERY  translate English to Sumerian: the first-born son
TARGET dumu-sag
PRE

## Save to Huggingface

In [38]:
model_path = os.path.abspath(f"/home/fak/nn/Data/generated/cuneiform/{model_id}-fullytrained")
trainer.save_model(model_path)
model_path

Saving model checkpoint to /home/fak/nn/Data/generated/cuneiform/t5-base-bi-p-akksux-en-20220723-023520-fullytrained
Configuration saved in /home/fak/nn/Data/generated/cuneiform/t5-base-bi-p-akksux-en-20220723-023520-fullytrained/config.json
Model weights saved in /home/fak/nn/Data/generated/cuneiform/t5-base-bi-p-akksux-en-20220723-023520-fullytrained/pytorch_model.bin
tokenizer config file saved in /home/fak/nn/Data/generated/cuneiform/t5-base-bi-p-akksux-en-20220723-023520-fullytrained/tokenizer_config.json
Special tokens file saved in /home/fak/nn/Data/generated/cuneiform/t5-base-bi-p-akksux-en-20220723-023520-fullytrained/special_tokens_map.json


'/home/fak/nn/Data/generated/cuneiform/t5-base-bi-p-akksux-en-20220723-023520-fullytrained'

In [39]:
tokenizer.save_pretrained(model_path)

tokenizer config file saved in /home/fak/nn/Data/generated/cuneiform/t5-base-bi-p-akksux-en-20220723-023520-fullytrained/tokenizer_config.json
Special tokens file saved in /home/fak/nn/Data/generated/cuneiform/t5-base-bi-p-akksux-en-20220723-023520-fullytrained/special_tokens_map.json


('/home/fak/nn/Data/generated/cuneiform/t5-base-bi-p-akksux-en-20220723-023520-fullytrained/tokenizer_config.json',
 '/home/fak/nn/Data/generated/cuneiform/t5-base-bi-p-akksux-en-20220723-023520-fullytrained/special_tokens_map.json',
 '/home/fak/nn/Data/generated/cuneiform/t5-base-bi-p-akksux-en-20220723-023520-fullytrained/tokenizer.json')